# Predicting Telco Customer Churn Using Random Forest and Gradient Boosting

In this notebook, we will use ensemble machine learning methods – **Random Forest** and **Gradient Boosting** – to build predictive models for Telco Customer Churn. We will go through dataset exploration, preprocessing, model training, evaluation, and analysis. This project demonstrates how ensemble methods can improve prediction performance over individual models.

## 1. Introduction

Customer churn is a critical problem for telecom companies. Accurately predicting churn offers actionable insights to improve customer retention and reduce costs. 

We use two popular ensemble methods in this project:

- **Random Forest:** Combines multiple decision trees trained on random subsets of data and features (bagging) to reduce variance and avoid overfitting.
- **Gradient Boosting:** Sequentially builds decision trees that each correct the errors of its predecessors, focusing on the misclassified examples.

Both methods have advantages for handling complex and noisy datasets and are widely used in real-world predictive modeling.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve)
from sklearn.preprocessing import StandardScaler

# For SHAP analysis
import shap

In [ ]:
# Configure plots
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# This module is used to handle warning messages in Python
import warnings
warnings.filterwarnings('ignore')

## 2. Dataset Description & Exploratory Data Analysis (EDA)

We use the Telco Customer Churn dataset. The dataset includes customers’ demographic, account, and service data along with whether they churned. Some of the key features include:

- `gender`, `SeniorCitizen`, `Partner`, `Dependents`
- `tenure`, `PhoneService`, `MultipleLines`, 
- `InternetService`, `OnlineSecurity`, `TechSupport`, etc.
- `MonthlyCharges`, `TotalCharges`, and the target variable `Churn`

Let’s load the dataset and perform some initial exploration.

In [ ]:
# Load the dataset from a public URL
data_url = "https://huggingface.co/KawgKawgKawg/Telephone-Company-Churn-Classification-Model/raw/main/Telco-Customer-Churn.csv"
df = pd.read_csv(data_url)

### Preview of the Dataset

Let's look at the first few rows of the dataset to get a sense of the data.

In [ ]:
# Display basic information
print('Dataset shape:', df.shape)
df.head()

In [ ]:
# Quick overview of the dataset
print('Dataset info:')
df.info()

In [ ]:
# Generate descriptive statistics for the DataFrame 'df', including all columns and data types
print('\nSummary statistics:')
df.describe(include='all')

### Categorical Feature Distributions

Let's visualize the distribution of some key categorical features.

In [ ]:
# Plot distributions for several important categorical features
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService', 'Contract', 'PaymentMethod']

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, col in enumerate(categorical_features):
    sns.countplot(x=col, data=df, palette='colorblind', ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')

# Hide any unused subplots
for j in range(len(categorical_features), len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize the distribution of the target variable 'Churn'
plt.figure()
sns.countplot(x='Churn', data=df, palette='pastel')
plt.title('Distribution of Churn')
plt.show()

In [ ]:
# Distribution of Monthly Charges
plt.figure()
sns.histplot(df['MonthlyCharges'], kde=True, color='blue')
plt.title('Distribution of Monthly Charges')
plt.show()

## 3. Data Preprocessing

This section includes:

- **Handling missing values:** Converting and cleaning the `TotalCharges` column.
- **Encoding categorical variables:** Using one-hot encoding for non-numeric features.
- **Feature scaling:** Scaling numerical features using `StandardScaler`.
- **Train/Test split:** Dividing the data for training and evaluation.

Let's preprocess the data.

In [ ]:
# Convert 'TotalCharges' to numeric and drop rows with NaN values if any
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    df = df.dropna(subset=['TotalCharges'])

# Drop 'customerID' column since it doesn't doesn't have any meaningful information for prediction
df.drop('customerID', axis=1, inplace=True)

# Convert target variable 'Churn' to binary: Yes -> 1, No -> 0
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Check for remaining missing values
print('Missing values in each column:')
print(df.isnull().sum())

In [ ]:
# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop('Churn')
print('Categorical columns:', list(categorical_cols))
print('Numerical columns:', list(numerical_cols))

In [ ]:
# One-hot encoding for categorical variables (excluding the target if present)
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Scale numerical features
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

df_encoded.head()

In [ ]:
# Split the data into features (X) and target (y)
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print('Training set shape:', X_train.shape)
print('Testing set shape:', X_test.shape)

## 4. Mathematical Explanation

**Random Forest**:

- An ensemble of decision trees built using the bagging technique. 
- Each tree is trained on a bootstrapped subset of data with random feature selection, which reduces variance and overfitting.
- Final prediction is typically made by aggregating the predictions from individual trees (majority vote for classification).

**Gradient Boosting**:

- Builds models sequentially; each new tree attempts to correct errors made by the previous ensemble.
- Uses gradient descent to minimize a loss function, thereby focusing on “difficult” cases.
- Tends to have lower bias but may be more prone to overfitting if not properly regularized.

**Key Differences:**

- **Random Forest:** Parallel training, less sensitive to hyperparameters, good for reducing overfitting.
- **Gradient Boosting:** Sequential training, often achieves higher accuracy but requires careful tuning of hyperparameters (learning rate, number of trees, etc.).

## 5. Model Training & Evaluation

We'll now train two models:

- A **Random Forest Classifier**
- A **Gradient Boosting Classifier**

We will perform hyperparameter tuning using grid search (with a minimal grid for demonstration) and compare their performance using metrics such as accuracy, precision, recall, F1-score, and ROC-AUC.

Let's get started with model training.

### 5.1 Random Forest Classifier

In [ ]:
# Define a parameter grid
rf_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 3, 5, 10, 20]
}

# Set up GridSearchCV
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
rf_grid.fit(X_train, y_train)

print('Best parameters for Random Forest:', rf_grid.best_params_)

# Best estimator from grid search
rf_model = rf_grid.best_estimator_
rf_model

In [ ]:
# Evaluate on test data
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print('\nRandom Forest Evaluation Metrics:')
print('Accuracy:', accuracy_score(y_test, rf_pred))
print('Precision:', precision_score(y_test, rf_pred))
print('Recall:', recall_score(y_test, rf_pred))
print('F1 Score:', f1_score(y_test, rf_pred))
print('ROC-AUC:', roc_auc_score(y_test, rf_pred_proba))

In [ ]:
# Classification Report
print('\nClassification Report:')
print(classification_report(y_test, rf_pred, zero_division=0))

In [ ]:
# Confusion Matrix
cm_rf = confusion_matrix(y_test, rf_pred)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### 5.2 Gradient Boosting Classifier

In [ ]:
# Define a parameter grid
gb_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 10, 20]
}

# Set up GridSearchCV
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42), gb_param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
gb_grid.fit(X_train, y_train)

print('Best parameters for Gradient Boosting:', gb_grid.best_params_)

# Best estimator
gb_model = gb_grid.best_estimator_
gb_model

In [ ]:
# Evaluate on test data
gb_pred = gb_model.predict(X_test)
gb_pred_proba = gb_model.predict_proba(X_test)[:, 1]

print('\nGradient Boosting Evaluation Metrics:')
print('Accuracy:', accuracy_score(y_test, gb_pred))
print('Precision:', precision_score(y_test, gb_pred))
print('Recall:', recall_score(y_test, gb_pred))
print('F1 Score:', f1_score(y_test, gb_pred))
print('ROC-AUC:', roc_auc_score(y_test, gb_pred_proba))

In [ ]:
print('\nClassification Report:')
print(classification_report(y_test, gb_pred, zero_division=0))

In [ ]:
# Confusion Matrix
cm_gb = confusion_matrix(y_test, gb_pred)
sns.heatmap(cm_gb, annot=True, fmt='d', cmap='Greens')
plt.title('Gradient Boosting Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### Model Comparison Table

Let's compare the performance of Random Forest and Gradient Boosting side by side.

> **Note:** The parameter grids used for hyperparameter tuning were:
> - Random Forest: `n_estimators` = [50, 100, 200, 300], `max_depth` = [None, 3, 5, 10, 20] (20 combinations)
> - Gradient Boosting: `n_estimators` = [50, 100, 200, 300], `learning_rate` = [0.01, 0.05, 0.1], `max_depth` = [3, 5, 10, 20] (48 combinations; `None` removed for compatibility)

In [ ]:
# Collect metrics for both models
metrics = {
    "Model": ["Random Forest", "Gradient Boosting"],
    "Accuracy": [
        accuracy_score(y_test, rf_pred),
        accuracy_score(y_test, gb_pred)
    ],
    "Precision": [
        precision_score(y_test, rf_pred, zero_division=0),
        precision_score(y_test, gb_pred, zero_division=0)
    ],
    "Recall": [
        recall_score(y_test, rf_pred, zero_division=0),
        recall_score(y_test, gb_pred, zero_division=0)
    ],
    "F1 Score": [
        f1_score(y_test, rf_pred, zero_division=0),
        f1_score(y_test, gb_pred, zero_division=0)
    ],
    "ROC-AUC": [
        roc_auc_score(y_test, rf_pred_proba),
        roc_auc_score(y_test, gb_pred_proba)
    ]
}

comparison_df = pd.DataFrame(metrics)
comparison_df.set_index("Model", inplace=True)
comparison_df

### ROC Curve Comparison

Let's compare the ROC curves of both models on the same plot for a direct visual comparison.

In [ ]:
# Compute ROC curves
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_pred_proba)
fpr_gb, tpr_gb, _ = roc_curve(y_test, gb_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_score(y_test, rf_pred_proba):.2f})', color='#0072B2')
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {roc_auc_score(y_test, gb_pred_proba):.2f})', color='#D55E00')
plt.plot([0, 1], [0, 1], 'k--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

## 6. Model Analysis & Visualization

In this section, we analyze the models using feature importance and SHAP values for model interpretability.

- **Feature Importance:** We plot the (normalized) importance scores for both models.
- **SHAP Values:** SHAP (SHapley Additive exPlanations) provides insights into how each feature contributes to individual predictions.

### SHAP Analysis for Model Interpretability

We use SHAP (SHapley Additive exPlanations) to interpret our models. SHAP values help us understand how each feature contributes to the model's predictions for individual samples.

- **Summary Plot (Bar):** Shows the average absolute SHAP value for each feature, indicating overall feature importance.
- **Summary Plot (Dot):** Shows the distribution of SHAP values for each feature across samples, revealing both importance and the direction of impact.

> **Note:** For binary classification, SHAP returns a single array of values. For multi-class, it returns a list of arrays (one per class). Here, we use the array directly for binary classification.

Let's first plot the feature importances and SHAP Values from the Random Forest model.

In [ ]:
# Feature importance from the Random Forest model
rf_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
rf_importances = rf_importances.sort_values(ascending=False)[:15]  # Top 15 features

plt.figure()
rf_importances.plot(kind='bar')
plt.title('Top 15 Feature Importances from Random Forest')
plt.ylabel('Importance Score')
plt.show()

In [ ]:
# Initialize the TreeExplainer
rf_explainer = shap.TreeExplainer(rf_model)

# Compute SHAP values for a subset of the test data
rf_shap_values = rf_explainer.shap_values(X_test.iloc[:100])

# Plot the SHAP summary plot
shap.initjs()
shap.summary_plot(rf_shap_values[1], X_test.iloc[:100], plot_type='bar')

Plot the feature importances and SHAP Values from the Gradient Boosting model.

In [ ]:
# Feature importance from the Gradient Boosting model
gb_importances = pd.Series(gb_model.feature_importances_, index=X.columns)
gb_importances = gb_importances.sort_values(ascending=False)[:15]  # Top 15 features

plt.figure()
gb_importances.plot(kind='bar')
plt.title('Top 15 Feature Importances from Gradient Boosting')
plt.ylabel('Importance Score')
plt.show()

In [ ]:
# Initialize the TreeExplainer
gb_explainer = shap.TreeExplainer(gb_model)

# Compute SHAP values for a subset of the test data
gb_shap_values = gb_explainer.shap_values(X_test.iloc[:100])

# Plot the SHAP summary plot
shap.initjs()
shap.summary_plot(gb_shap_values, X_test.iloc[:100], plot_type='bar')

## 7. Discussion

From the results above, we can note the following:

- Both Random Forest and Gradient Boosting achieved good performance in predicting customer churn.
- The ROC curves and evaluation metrics (accuracy, precision, recall, F1, ROC-AUC) offer insights into the trade-offs between the models. 
- Feature importance and the SHAP values help us understand which features most strongly impact the prediction. For instance, features related to monthly charges and tenure often prove to be important.

While Gradient Boosting sometimes achieves slightly higher ROC-AUC, it can be more sensitive to hyperparameter choices and may require careful tuning. On the other hand, Random Forest tends to be more robust but might not capture all patterns as finely as boosting.

## 8. Conclusion

In this project, we:

- Explored and preprocessed the Telco Customer Churn dataset.
- Built and tuned predictive models using both Random Forest and Gradient Boosting.
- Evaluated and compared their performance using multiple metrics and visualized the model interpretation results using feature importances and SHAP plots.

The insights gained can help telecom companies to proactively identify and address factors that contribute to customer churn. Future work may include:

- Further hyperparameter tuning and cross-validation.
- Handling class imbalances through methods such as SMOTE.
- Exploring additional ensemble methods or deep learning techniques.
- Integrating the model into a real-time deployment pipeline.

## 9. Additional Considerations

- **Handling Class Imbalance:** In real-world applications, further techniques (like SMOTE or class-weighting) might be needed if the dataset is highly imbalanced.
- **Model Optimization:** Additional parameters and cross-validation strategies can be applied to get the best performance.
- **Deployment:** The trained model can be deployed using REST APIs, cloud services, or integrated into business applications for real-time predictions.

This notebook provides a framework that can be extended for other applications such as credit scoring or product recommendation systems.


## 10. References

1. [Random Forests](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm) by Leo Breiman.
2. [Gradient Boosting Machine](https://research.google/pubs/pub49317/) by Friedman, J. H.
3. [Telco Customer Churn Dataset](https://huggingface.co/KawgKawgKawg/Telephone-Company-Churn-Classification-Model/raw/main/Telco-Customer-Churn.csv) - Public dataset available on Hugging Face.
4. [SHAP Documentation](https://github.com/slundberg/shap) - For model interpretability.